# Script to compute Panoptic Quality (PQ) metric for each image and to store results in an excel file


# Created by Ruchika Verma

This code will generate an excel file with image names in rows and respective PQ metrics in column 1

Note: This code will work if n-ary masks are stored in both ground truth and predicted path.

The format to save predicted masks is given below :
Folder -> Patient name
Sub-folder -> Sub-images under each patient
Sub-Sub-folder -> Annotated cell-type on each sub-image which contains n-ary masks

Please run n-ary mask generation code from the link below to see format to save ground truth and predicted masks
https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/n-ary_mask_generation.ipynb

# Input
ground_truth_path: #Ground truth path to read data from
Predicted_path: ' #Path to read predicted outcomes from

# Output
An excel file with name 'PQ_metric.xls' will store on the given ground_truth_path

# Reference
Panoptic Segmentation
Alexander Kirillov, Kaiming He, Ross Girshick, Carsten Rother and Piotr Dollár
arXiv:1801.00868, 2018.

In [1]:
import os
import numpy as np
import glob
import cv2
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage
import xlwt 
from xlwt import Workbook 

In [2]:
# Compute Panoptic quality metric for each image
def Panoptic_quality(ground_truth_image,predicted_image):
    TP = 0
    FP = 0
    FN = 0
    sum_IOU = 0
    matched_instances = {}# Create a dictionary to save ground truth indices in keys and predicted matched instances as velues
                        # It will also save IOU of the matched instance in [indx][1]

    # Find matched instances and save it in a dictionary
    for i in np.unique(ground_truth_image):
        if i == 0:
            pass
        else:
            temp_image = np.array(ground_truth_image)
            temp_image = temp_image == i
            matched_image = temp_image * predicted_image
        
            for j in np.unique(matched_image):
                if j == 0:
                    pass
                else:
                    pred_temp = predicted_image == j
                    intersection = sum(sum(temp_image*pred_temp))
                    union = sum(sum(temp_image + pred_temp))
                    IOU = intersection/union
                    if IOU> 0.5:
                        matched_instances [i] = j, IOU 
                        
    # Compute TP, FP, FN and sum of IOU of the matched instances to compute Panoptic Quality               
                        
    pred_indx_list = np.unique(predicted_image)
    pred_indx_list = np.array(pred_indx_list[1:])

    # Loop on ground truth instances
    for indx in np.unique(ground_truth_image):
        if indx == 0:
            pass
        else:
            if indx in matched_instances.keys():
                pred_indx_list = np.delete(pred_indx_list, np.argwhere(pred_indx_list == [indx][0]))
                TP = TP+1
                sum_IOU = sum_IOU+matched_instances[indx][1]
            else:
                FN = FN+1
    FP = len(np.unique(pred_indx_list))
    PQ = sum_IOU/(TP+0.5*FP+0.5*FN)
    
    return PQ

In [3]:
ground_truth_path = 'D:\MoNuSAC_ground_truth_masks' #Ground truth path to read data from
Predicted_path = 'D:\MoNuSAC_predicted_masks' #Path to read predicted outcomes from

import os
os.chdir(ground_truth_path)

In [4]:
files=glob.glob('./**/**/**/*.tif')

In [5]:
# Workbook is created 
wb = Workbook() 

ccbt = wb.add_sheet('PQ metric') 
ccbt.write(0, 0, 'Patient ID')
ccbt.write(0, 1, 'Panoptic Quality')

for image_count,filei in enumerate(files):
    print(filei)
    ground_truth_image = np.array(Image.open(filei))
    predicted_image = np.array(Image.open(Predicted_path+filei))
    PQ = Panoptic_quality(ground_truth_image,predicted_image)
    ccbt.write(image_count+1,0, filei)#Add image name in excel file
    ccbt.write(image_count+1,1, PQ)

    print(PQ)
wb.save('PQ_metric.xls')    #Save data in an excel file titled as PQ_metric

.\TCGA-55-1594-01Z-00-DX1\TCGA-55-1594-01Z-00-DX1_001\Epithelial\1_mask.tif
0.88
.\TCGA-86-7713-01Z-00-DX1\TCGA-86-7713-01Z-00-DX1_004\Epithelial\109_mask.tif
0.66
.\TCGA-86-7713-01Z-00-DX1\TCGA-86-7713-01Z-00-DX1_004\Lymphocyte\110_mask.tif
0.71
.\TCGA-86-8672-01Z-00-DX1\TCGA-86-8672-01Z-00-DX1_1\Lymphocyte\114_mask.tif
1.0
